In [ ]:
import pandas as pd
import mysql.connector as mysql
from getpass import getpass
import numpy as np
import sys

In [ ]:
while True:
    username = getpass('Please enter MYSQL user\n')
    pass_word = getpass('Please enter MYSQL password\n')
    try:
        db = mysql.connect(host='localhost', user= username, password= pass_word)
        print('database successfully connected')
        break
    except:
        print('database connection not successfull.')
        i = input('Do you want to try again? (Y/N)\n')
        if i.lower == 'n':
            sys.exit('Thank you for using this program')

cur = db.cursor()

In [ ]:
cur.execute('USE yt_history')
query = 'SELECT * FROM video'
cur.execute(query)
history_data = pd.DataFrame(cur.fetchall(),columns = ['Title','Channel','Category','Time','Date'])

In [ ]:
history_data.index = np.arange(1,len(history_data)+1)

In [ ]:
history_data.index.name = 'Sr_no'

In [ ]:
df = history_data.groupby(["Date","Category"],sort = True)

In [ ]:
new_df = df.size()

In [ ]:
plot_data = new_df.unstack()

In [ ]:
import plotly.graph_objects as go
import chart_studio.plotly as py
import seaborn as sns
import cufflinks as cf
%matplotlib inline
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected = True)
cf.go_offline

In [ ]:
fig = go.Figure()
colors = ['rgb(255, 53, 94)','rgb(253, 91, 120)','rgb(255, 96, 55)','rgb(255, 153, 102)','rgb(255, 153, 51)','rgb(255, 204, 51)','rgb(255, 255, 102)','rgb(204, 255, 0)','rgb(102, 255, 102)','rgb(170, 240, 209)','rgb(22, 208, 203)','rgb(80, 191, 230)','rgb(156, 39, 176)','rgb(238, 52, 210)','rgb(255, 0, 204)']
x=0
for (category, column) in plot_data.items():
    fig.add_trace(go.Scatter(x=plot_data.index, y=column,name=category,mode='lines+markers',line = {'width':1.34},hovertemplate=f'Category: {category}' + '<br>Date: %{x}<br>No. Of Videos: %{y}',line_color=colors[x]))
    x+=1

In [ ]:
fig.update_layout(title = dict(text='Video Consumption Data Over Time By Category',font = dict(size=24,color='black'),xanchor='center',x=0.5,y=0.9,yanchor='top'))
fig.update_layout(plot_bgcolor='#f2f6fa',paper_bgcolor='#f2f6fa')
fig.update_layout(xaxis=dict(title=dict(text='Date',font=dict(size=18,color='black')),showline=True,linecolor='white',linewidth=4,showgrid=False,tickfont=dict(size=13,color='black')))
fig.update_layout(yaxis=dict(title=dict(text='No. Of Videos',font=dict(size=18,color='black')),showgrid=False,zeroline=False,tickfont=dict(size=13,color='black')),showlegend=True)
fig.show()

In [ ]:
count_data = history_data['Category'].value_counts()

In [ ]:
count_df = pd.DataFrame(count_data)

In [ ]:
count_df.rename(columns={'Category':'No. Of Videos'},inplace=True)

In [ ]:
count_df['Category'] = count_df.index

In [ ]:
count_df.index = np.arange(1,len(count_df)+1)

In [ ]:
new_fig = px.bar(count_df,x='Category',y='No. Of Videos', title='VIDEO CONSUMPTION DATA BY CATEGORY',color = 'Category',text='No. Of Videos')
new_fig.update_traces(textposition='outside',hovertemplate='Category: %{x}<br>No. Of Videos: %{y}')
new_fig.update_layout(title = dict(text='Video Consumption Data By Category',font = dict(size=24,color='black'),xanchor='center',x=0.5,y=0.9,yanchor='top'))
new_fig.update_layout(plot_bgcolor='#f2f6fa',paper_bgcolor='#f2f6fa')
new_fig.update_layout(xaxis=dict(title=dict(text='Category',font=dict(size=18,color='black')),showline=True,linecolor='white',linewidth=4,showgrid=False,tickfont=dict(size=13,color='black')))
new_fig.update_layout(yaxis=dict(title=dict(text='No. Of Videos',font=dict(size=18,color='black')),showgrid=False,zeroline=False,tickfont=dict(size=13,color='black')),showlegend=True)
new_fig.show()

In [ ]:
frames = []
x_axis = list(plot_data.columns)
def y_axis(date):
    return list(plot_data.loc[date])

sliders_dict= {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Date:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}
for date in plot_data.index:
    frame = {'data':[go.Bar(x=x_axis,y=y_axis(date),textposition='outside',hovertemplate='Category: %{x}<br>No. Of Videos: %{y}',marker=dict(color=colors))],'name':date}
    frames.append(frame)
    slider_step = {"args": [
        [date],
        {"frame": {"duration": 300, "redraw": False},
         "mode": "immediate",
         "transition": {"duration": 300}}
    ],
        "label": date,
        "method": "animate"}
    sliders_dict["steps"].append(slider_step)
anim_fig = go.Figure(
    data=[go.Bar(x=x_axis,y=y_axis('2023-01-28'),textposition='outside',hovertemplate='Category: %{x}<br>No. Of Videos: %{y}',marker=dict(color=colors))], 
    layout= dict(title = dict(text='Video Consumption Data By Category Over Time',font = dict(size=24,color='black'),xanchor='center',x=0.5,y=0.9,yanchor='top'),xaxis=dict(showline=True,linecolor='white',linewidth=4,showgrid=False,tickfont=dict(size=13,color='black')),yaxis=dict(title=dict(text='No. Of Videos',font=dict(size=18,color='black')),showgrid=False,zeroline=False,range=[0,250],tickfont=dict(size=13,color='black'))),
    frames= frames)
anim_fig["layout"]["updatemenus"] = [
    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 500, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 300,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
]
anim_fig["layout"]["sliders"] = [sliders_dict]

In [ ]:
anim_fig.show()